In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

url = 'https://fairytalez.com/fairy-tales/'
session = requests.Session()
response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(response.text, "html5lib")

## Get all the urls of fairy tales

In [2]:
story_index = [story.find_all('a') for story in soup.find_all('div', class_='azindex')]
all_links = []
for index in story_index:
    for links in index:
        all_links.append(links.get('href'))

In [3]:
len(all_links)

3177

## Getting all the fairy tales

In [9]:
df = pd.DataFrame(columns = ['title','author','region','body'])
count = 0

for url in all_links:
    story_url = url
    session = requests.Session()
    response = session.get(story_url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html5lib")
    title = soup.find('h1', class_= 'title entry-title').text
    author = soup.find('span', class_= 'author vcard').find('span', class_ = 'categories').text
    region = soup.find('div', class_= 'author-name').find_all('span')[-1].text.strip()
    content = []
    for paras in soup.find('section', class_= 'entry').find_all('p'):
        paras = paras.text
        
        # remove in-text questions
        if re.match(r'^\d+\. *.+\? *$', paras):
            next
        
        # remove paragraphs of notes 
        if re.match(r'(Note:).*', paras):
            next 
        else:
            content.append(paras)
    body = ' '.join(content)
    
    df = df.append({'title': title,
                   'author': author,
                   'region': region,
                   'body': body},ignore_index=True)

In [10]:
df.head(10)

,title,author,region,body
0,A Clever Thief,Hindu Tales from the Sanskrit,Indian,"Chapter I A certain man, named Hari-Sarman, wh..."
1,A Lac of Rupees for a Piece of Advice,Joseph Jacobs,Indian,A poor blind Brahman and his wife were depende...
2,A Leaf from the Sky,Hans Christian Andersen,Danish Nordic Scandinavian,"High up yonder, in the thin clear air, flew an..."
3,A Legend of Confucius,The Chinese Fairy Book,Chinese,"When Confucius came to the earth, the Kilin, t..."
4,A Lesson for Kings,Joseph Jacobs,Indian,"Once upon a time, when Brahma-datta was reigni..."
5,A Royal Thief-Catcher,Hindu Tales from the Sanskrit,Indian,Chapter I In one of the smaller cities of Indi...
6,A Story from the Sand-Dunes,Hans Christian Andersen,Danish Nordic Scandinavian,This is a story from the sand-dunes or sand-hi...
7,Admetus and Alcestis,James Baldwin,Greek,Part I: The Slave IN a little town north of De...
8,Adventures of Bull-Turns-Round,George Bird Grinnell,Native American North American,"Once the camp moved, but one lodge stayed. It ..."
9,Aegir's Feast,Foster & Cummings,Nordic Scandinavian,"I. Ægir was the ruler of the ocean, and his ho..."


37 stories are too difficult to scrape because each page has different html structure. They are not included in the dataset.

In [11]:
df = df[df.body != '']

One missing region information. Checked author and it should be German

In [12]:
df.loc[(df.region == ''), 'region'] = 'German'

In [14]:
df.to_csv('fairytales.csv')